In [2]:
pip install azureml-sdk==1.59.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show azureml-sdk

Name: azureml-sdk
Version: 1.59.0
Summary: Used to build and run machine learning workflows upon the     Azure Machine Learning service.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: 
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/automl-env2/lib/python3.9/site-packages
Requires: azureml-core, azureml-dataset-runtime, azureml-pipeline, azureml-train-automl-client, azureml-train-core
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install 'azureml-sdk[notebooks]'

  Using cached azureml_widgets-1.59.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached azureml_contrib_notebook-1.59.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached papermill-2.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached jupyter_client-7.4.9-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipywidgets-7.8.5-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl.metadata (755 bytes)
  Using cached widgetsnbextension-3.6.10-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached jupyterlab_widgets-1.1.11-py3-none-any.whl.metadata (3.7 kB)
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached bleach-6.2.0-py3-none-any.whl.metadata (30 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached jupyterlab_pygments-0.3.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached mistune-3.1.3-py3-none-any.whl.metadata 

In [5]:
pip install azureml-sdk[automl]

  Using cached azureml_train_automl-1.59.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached azureml_automl_runtime-1.59.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached azureml_train_automl_runtime-1.59.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached azureml_responsibleai-1.59.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached azureml_training_tabular-1.59.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl.metadata (14 kB)
  Using cached onnxmltools-1.11.2-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached azureml_interpret-1.59.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached azureml_mlflow-1.59.0.post1-py3-none-any.whl.metadata (2.6 kB)
  Using cached protobuf-5.28.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached azureml_train_automl-1.59.0-py3-none-any.whl (2.4 kB)
Using cached azureml_automl_runtime-1.59.0-py3-none-any.whl (1.8 MB)
Using cached onnxmltools-1.11.2-py2.py3-none-any.whl (322 kB)
Using 

In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="hd")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-280210
Azure region: eastus2
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-280210


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Using existing cluster")
except ComputeTargetException:
    print("No existing cluster found, creating a cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_nodes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Using existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                script='train.py',
                compute_target='Cluster',
                environment=sklearn_env)

#primary_metric = 'Accuracy'
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=20,
                max_concurrent_runs=4)

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from collections.abc import Mapping
from azureml.widgets import RunDetails


hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_483f4006-c6a4-48b3-b082-120c617b8b96
Web View: https://ml.azure.com/runs/HD_483f4006-c6a4-48b3-b082-120c617b8b96?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-280210/workspaces/quick-starts-ws-280210&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-05-07T15:23:57.9854820Z][GENERATOR][WARNING]Space size : 12 is less than max total jobs : 20, only 12 jobs will be generated 
[2025-05-07T15:23:59.7725332Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-05-07T15:24:00.2396276Z][SCHEDULER][INFO]Scheduling job, id='HD_483f4006-c6a4-48b3-b082-120c617b8b96_2' 
[2025-05-07T15:24:00.2120121Z][SCHEDULER][INFO]Scheduling job, id='HD_483f4006-c6a4-48b3-b082-120c617b8b96_0' 
[2025-05-07T15:24:00.3804072Z][SCHEDULER][INFO]Scheduling job, id='HD_483f4006-c6a4-48b3-b082-120c617b8b96_3' 
[2025-05-07T15:24:00.2405479Z][SCHEDULER][INFO]Scheduling job, id='HD_483f4006-c6a4-48b3-b082-120c617b8b96_1' 
[2025-05-

{'runId': 'HD_483f4006-c6a4-48b3-b082-120c617b8b96',
 'target': 'Cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-05-07T15:23:57.642609Z',
 'endTimeUtc': '2025-05-07T15:43:06.773389Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1695c7a8-0e0d-4f48-bc28-d9d5c17dc02e',
  'user_agent': 'python/3.9.21 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_483f4006-c6a4-48b3-b082-120c617b8b96_9',
  'score': '0.9141781985918912',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_483f4006-c6a4-48b3-b082-120c617b8b96_9'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train',

In [25]:
RunDetails(hyperdrive_run).show

<bound method _WidgetRunDetailsBase.show of <azureml.widgets._hyperdrive._run_details._HyperDriveRunDetails object at 0x7736b7ce97c0>>

In [10]:
#import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run.register_model(model_name = 'hd_best_model.pkl', model_path='outputs/')

Run(Experiment: hd,
Id: HD_483f4006-c6a4-48b3-b082-120c617b8b96_9,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-280210', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-280210'), name=hd_best_model.pkl, id=hd_best_model.pkl:1, version=1, tags={}, properties={})

In [11]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '1', '--max_iter', '100']


In [12]:
print(best_run.get_metrics())

{'Max iterations:': 100, 'Regularization Strength:': 1.0, 'Accuracy': 0.9141781985918912}


In [13]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [15]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)

In [17]:
# Submit your automl run
from azureml.core.experiment import Experiment

experiment = Experiment(ws, 'project1automl')
automl_run = experiment.submit(config=automl_config, show_out=True)

Experiment,Id,Type,Status,Details Page,Docs Page
project1automl,AutoML_1c832b4d-4507-432f-829e-9a7cf90d8b61,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2025/05/07 15:47:43 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/05/07 15:47:43 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - inference-schema (current: uninstalled, required: inference-schema)
 - xgboost (current: uninstalled, required: xgboost<=1.5.2)
 - prophet (current: uninstalled, required: prophet==1.1.4)
 - azureml-defaults (current: uninstalled, required: azureml-defaults)
 - pytorch-transformers (current: uninstalled, required: pytorch-transformers==1.0.0)
 - spacy (current: uninstalled, required: spacy==3.7.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [19]:
# Retrieve and save your best automl model.
from azureml.core.experiment import Experiment
from azureml.train.automl.run import AutoMLRun

best_run, fitted_model = automl_run.get_output()
best_run.register_model(model_name = 'automl_best_model.pkl', model_path='outputs/')

print(best_run)

Run(Experiment: project1automl,
Id: AutoML_1c832b4d-4507-432f-829e-9a7cf90d8b61_30,
Type: None,
Status: Completed)


In [20]:
print(best_run.get_details())

{'runId': 'AutoML_1c832b4d-4507-432f-829e-9a7cf90d8b61_30', 'target': 'local', 'status': 'Completed', 'startTimeUtc': '2025-05-07T16:06:35.461714Z', 'endTimeUtc': '2025-05-07T16:07:21.566876Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'project1automl\',\'compute_target\':\'local\',\'subscription_id\':\'9a7511b8-150f-4a58-8528-3e7d50216c31\',\'region\':\'eastus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_1c832b4d-4507-432f-829e-9a7cf90d8b61_30","experiment_name":null,"workspace_name":"quick-starts-ws-280210","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_g

In [22]:
print(best_run.get_metrics())

{'recall_score_macro': 0.7488284885110852, 'f1_score_weighted': 0.9127747531458221, 'balanced_accuracy': 0.7488284885110852, 'AUC_macro': 0.945653319682512, 'recall_score_weighted': 0.9166919575113809, 'recall_score_micro': 0.9166919575113809, 'AUC_micro': 0.9800359214425682, 'precision_score_macro': 0.8007846657656412, 'f1_score_micro': 0.9166919575113809, 'matthews_correlation': 0.5470646982446183, 'average_precision_score_micro': 0.9808617029298986, 'f1_score_macro': 0.7712559301517059, 'precision_score_weighted': 0.910664511422509, 'norm_macro_recall': 0.4976569770221704, 'AUC_weighted': 0.945653319682512, 'precision_score_micro': 0.9166919575113809, 'weighted_accuracy': 0.958390262021235, 'average_precision_score_weighted': 0.9537782699223901, 'accuracy': 0.9166919575113809, 'log_loss': 0.193263085735034, 'average_precision_score_macro': 0.8182651348982937, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_1c832b4d-4507-432f-829e-9a7cf90d8b61_30/confusion_matrix', 'a

In [23]:
cluster.delete()